In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
def EF_CH4(Bo, MCFj, Uj, Tij):
    """
    Emission factor for CH4 from wastewater treatment plants based on income group and treatment type.
    Source: IPCC 2006
    units requered:
        - Bo: [kg CH4 / kg BOD]
        - MCFj: fraction
        - Uj: fraction
        - Tij: fraction
    """
    return Bo*MCFj*Uj*Tij

In [3]:
def TOWi(Pi, BOD, I):
    """
    Total organic waste generated by the population.
    Source: IPCC 2006
    units requered:
        - Pi: [person]
        - BOD: [g/person/day]
        - I: unitless
    """
    return Pi*BOD*I*365

In [4]:
def EF_N2O(protein, Fnpr, F_non_con, F_ind_com):
    """
    Emission factor for N2O based on default values.
    Source: IPCC 2006
    units requered:
        - protein: [kg protein / person / yr]
        - Fnpr: [kg N / kg protein]
        - F_non_con: unitless
        - F_ind_com: unitless
    """
    return protein*Fnpr*F_non_con*F_ind_com

In [5]:
income_group_dic = {
    'rural': {
        'urbanization': 0.16,
        'sewer': 0.10
    },
    'urban_high_income': {
        'urbanization': 0.25,
        'sewer': 0.80
    },
    'urban_low_income': {
        'urbanization': 0.59,
        'sewer': 0.40
    }
}

We need to read two "different" formats. 
They have the same column names and the same structure at the beginning, the problem is that the second ones have some "remarks" at the end of each table and we need to remove that information before concatenating all the files.

In [6]:
paths = glob.glob(f'./Plan*L*.xls')

df1 = pd.DataFrame()

for path in paths:
    tmp = pd.read_excel(path, skiprows=11)

    tmp.rename(columns={'-': 'municipal_code', 
                    '-.1': 'municipality_name', 
                    '-.2': 'UF', 
                    '-.3': 'code1', 
                    '-.4': 'code2', 
                    '-.5': 'provedor_name', 
                    '-.6': 'sigla', 
                    '-.7': 'scope', 
                    '-.8': 'jur_nature', 
                    '-.9': 'service_type'}, inplace=True)

    # filter by the intereted columns
    tmp = tmp[['municipal_code', 'municipality_name', 'UF', 'service_type', 'GE005', 'ES001', 'ES005', 'ES006', 'ES014', 'ES015']]

    df1 = pd.concat([df1, tmp], ignore_index=True)

In [7]:
folder_path = './others/'

dataframes = []

for file in os.listdir(folder_path):
    if file.endswith('.xls'):  
        file_path = os.path.join(folder_path, file)

        df = pd.read_excel(file_path, skiprows=11)

        df.rename(columns={'-': 'municipal_code', 
                     '-.1': 'municipality_name', 
                     '-.2': 'UF', 
                     '-.3': 'code1', 
                     '-.4': 'code2', 
                     '-.5': 'provedor_name', 
                     '-.6': 'sigla', 
                     '-.7': 'scope', 
                     '-.8': 'jur_nature', 
                     '-.9': 'service_type'}, inplace=True)
        
        stop_index = df[df['municipal_code'] == 'TOTALIZAÇÃO DAS INFORMAÇÕES DESAGREGADAS (Desagre):  '].index.to_list()[0]

        if not pd.isna(stop_index):
            df = df.iloc[:stop_index]
        
        dataframes.append(df)

# Combine all processed DataFrames into one
df2 = pd.concat(dataframes, ignore_index=True)

# filter by the intereted columns
df2 = df2[['municipal_code', 'municipality_name', 'UF', 'service_type', 'GE005', 'ES001', 'ES005', 'ES006', 'ES014', 'ES015']]

In [8]:
# concatenate the two DataFrames
df_f = pd.concat([df1, df2], ignore_index=True)

In [9]:
# Select only the rows related to the sewer system information and one mucipality atended by the service
df_f = df_f[~df_f['service_type'].isin(['Água']) & (df_f['GE005'] == 1)]

In [10]:
# List of income group keys
income_groups = ['rural', 'urban_high_income', 'urban_low_income']

In [11]:
# Calculate population for each income group
for group in income_groups:
    df_f[f'{group}'] = df_f['ES001'] * income_group_dic[group]['urbanization']

In [22]:
# reformat of the DataFrame
df_f = df_f.melt(
    id_vars=['municipal_code', 'municipality_name', 'UF', 'ES005', 'ES006', 'ES014', 'ES015'], 
    value_vars=['rural', 'urban_high_income', 'urban_low_income'], 
    var_name='income_group', 
    value_name='population_by_income_group')

### CH4 calculations

In [23]:
## CH4 Calculation
df_f1 = df_f.copy()

In [24]:
# apply the total organic waste generation function to each income group
BOD = 50 # [g/ person / day]
I = 1.25
df_f1['TOWi'] = df_f1['population_by_income_group'].apply(TOWi, BOD= BOD, I=I)

In [25]:
# Bo [kg CH4 / kg BOD]
Bo = 0.6  
# Methane Correction Factor
MCFj = 0.1  

# calculate the emission factor for each income group
df_f1['emissionfactor_value'] = df_f1['income_group'].apply(
    lambda group: EF_CH4(
        Bo,
        MCFj,
        income_group_dic[group]['urbanization'],
        income_group_dic[group]['sewer']
    )
)

# assign the gas name and emission factor units
df_f1['gas_name'] = 'CH4'
df_f1['emissionfactor_units'] = 'kg/kg BOD'

In [26]:
# calculate the emissions value
df_f1['emissions_value_tmp'] = df_f1['emissionfactor_value']*df_f1['TOWi']

In [27]:
# create a column to store the metadata
df_f1["metadata"] = df_f1.apply(
    lambda row: {
        "activity_subcategory_type1": 'income_group',
        "activity_subcategory_typename1": row['income_group'],
        "activity_subcategory_type2": 'treatment_type',
        "activity_subcategory_typename2": 'sewer',
        "activity_subcategory_type3": 'TOWi',
        "activity_subcategory_typename3": row['TOWi']
    },
    axis=1,
)

In [28]:
df_f1.drop(columns=['income_group', 'TOWi'], inplace=True)

### N2O calculations

In [29]:
protein = 27.8     # [kg protein / person / yr]
Fnpr = 0.16        # [kg N / kg protein]
F_non_con = 1.4    # for countries with garbage disposals
F_ind_com = 1.25   # centralized systems

In [31]:
# emission factor calculation
df_f['emissionfactor_value'] = EF_N2O(protein, Fnpr, F_non_con, F_ind_com)

# emissions value calculation by income group
df_f['emissions_value_tmp'] = df_f['population_by_income_group'] * df_f['emissionfactor_value'] * 0.01 * 44.28

# assign the gas name and emission factor units
df_f['gas_name'] = 'N2O'
df_f['emissionfactor_units'] = 'kg/person'

In [32]:
# create a column to store the metadata
df_f["metadata"] = df_f.apply(
    lambda row: {
        "activity_subcategory_type1": 'income_group',
        "activity_subcategory_typename1": row['income_group'],
        "activity_subcategory_type2": 'treatment_type',
        "activity_subcategory_typename2": 'sewer'
    },
    axis=1,
)

In [33]:
df_f.drop(columns=['income_group'], inplace=True)

In [34]:
# final df
df_final = pd.concat([df_f, df_f1], ignore_index=True)

### Assignation scope

In [36]:
# calculate the fraction of each scope
df_final['fraction_scope1'] = (df_final['ES005']-df_final['ES015']-df_final['ES014'])/df_final['ES005']
df_final['fraction_scope3'] = df_final['ES015']/df_final['ES005']

In [37]:
# calculate the emissions for each scope
df_final['III.4.1'] = df_final['emissions_value_tmp']*df_final['fraction_scope1']
df_final['III.4.2'] = df_final['emissions_value_tmp']*df_final['fraction_scope3']

In [38]:
# reformating the DataFrame
df_final = df_final.melt(
    id_vars=['municipal_code', 'municipality_name', 'UF', 'population_by_income_group', 'emissionfactor_value', 'gas_name', 'metadata', 'emissionfactor_units'], 
    value_vars=['III.4.1', 'III.4.2'], 
    var_name='GPC_refno', 
    value_name='emissions_value')

In [39]:
# drop the rows with zero emissions
df_final = df_final[df_final['emissions_value'] != 0]

# drop the rows with NaN values
df_final.dropna(subset=['emissions_value'], inplace=True)

In [40]:
# rename the population column by income group as the activity value
df_final.rename(columns={'population_by_income_group': 'activity_value', 'municipality_name': 'actor_name'}, inplace=True)

In [41]:
# assign the activity units
df_final['activity_units'] = 'person'
df_final['activity_name'] = 'treatment-type-centralized-aerobic-treatment-plan-well-managed'

In [43]:
df_final.to_csv('emissions_waste.csv', index=False)